In [1]:
import os
os.environ["USE_PYGEOS"] = "0"  # suppress geopandas warning

In [2]:
import matplotlib.pyplot as plt
import osmnx as ox

# Download OSM network data and geometries

In [3]:
# 9 cities in NL
list_cities = [ "Arnhem", "Breda", "Nijmegen", "Delft", "Amersfoort", "Leiden", "Enschede", "Deventer", "Gouda"]
country = "NL"
crs_epsg = 28992  # RD

In [4]:
# retrieve nodes and edges
for city in list_cities:
    place = city + ", NL"
    print(city)

    G = ox.graph_from_place(place, network_type="walk")

    # extract nodes and edges
    nodes, edges = ox.graph_to_gdfs(G)

    # retrieve buildings
    tags = {'building': True}
    buildings = ox.geometries_from_place(place, tags)

    # retrieve parks
    tags = {'leisure': 'park', 'landuse': 'grass'}
    parks = ox.geometries_from_place(place, tags)

    # clean nodes
    nodes = nodes[["geometry"]]
    nodes = nodes.to_crs(crs_epsg)
    nodes["x"] = nodes.geometry.x
    nodes["y"] = nodes.geometry.y

    # clean edges
    edges = edges.reset_index()
    edges = edges[["u", "v", "key", "length", "geometry"]]
    edges = edges.to_crs(crs_epsg)

    # clean parks
    parks = parks.reset_index("element_type")
    parks = parks[parks["element_type"] == "way"]
    parks = parks[["geometry"]]
    parks = parks.to_crs(crs_epsg)

    # clean buildings
    buildings = buildings.reset_index("element_type")
    buildings = buildings[buildings["element_type"] == "way"]
    buildings = buildings[["geometry"]]
    buildings = buildings.to_crs(crs_epsg)

    # write output
    from pathlib import Path

    p_data = Path("./data/") / city
    p_data.mkdir(parents=False)     
    nodes.to_file(p_data / "nodes_{}.shp".format(city))
    edges.to_file(p_data / "edges_{}.shp".format(city))
    parks.to_file(p_data / "parks_{}.shp".format(city))
    buildings.to_file(p_data / "buildings_{}.shp".format(city))
    

Amersfoort
Leiden
Enschede
Deventer
Gouda
